In [ ]:
!pip install selenium
!pip install parsel

In [167]:
from selenium import webdriver
from parsel import Selector
import time 
import pandas as pd
from selenium.webdriver.common.by import By

In [ ]:
scrollable_div = driver.find_element(By.cssSelector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show'))
driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)

In [157]:
def scroll_down_page():
    SCROLL_PAUSE_TIME = 0.5
    while True:
        # Get scroll height
        ### This is the difference. Moving this *inside* the loop
        ### means that it checks if scrollTo is still scrolling 
        last_height = driver.execute_script("return document.body.scrollHeight")
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue

# Google maps scarping

In [170]:
def getall_restaurants(chromedrive_path,url):    
    all_restaurants = []
    driver = webdriver.Chrome(chromedrive_path)
    driver.get(url)
    #Scroll to Bottom of Webpage
    driver.maximize_window()
    #scrollable_div = driver.find_element(By.cssSelector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show'))
    #driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)    #scroll_down_page()
    #time.sleep(10)
    all_restaurants_content = driver.page_source
    all_restaurants_response = Selector(all_restaurants_content)
    for el in all_restaurants_response.xpath('//div[contains(@aria-label, "Результаты по запросу")]/div/div[./a]'):
        all_restaurants.append({
            'link': el.xpath('./a/@href').extract_first(''),
            'title': el.xpath('./a/@aria-label').extract_first('')
        })
    #driver.quit()
    return all_restaurants

In [171]:
chromedrive_path = './chromedriver' 
google_maps_ulr  = 'https://www.google.com/maps/search/%D0%A0%D0%B5%D1%81%D1%82%D0%BE%D1%80%D0%B0%D0%BD%D1%8B+%D1%81%D0%BF%D0%B1/@59.9549593,30.2748781,11z/data=!3m1!4b1'
restaurants_list = getall_restaurants(chromedrive_path,google_maps_ulr)

<ipython-input-170-5b7ffe797524>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [128]:
def get_resaurants_addresses(restaurants):
    restaurants_info = []
    driver = webdriver.Chrome(chromedrive_path)
    for rest in restaurants:
        #driver = webdriver.Chrome(chromedrive_path)
        rest_url = rest['link']
        driver.get(rest_url)
        rest_content = driver.page_source
        rest_response = Selector(rest_content)
        rest_address = rest_response.xpath('//button[contains(@aria-label, "Адрес:")]/@aria-label').extract_first('').split('Адрес: ')[1]        
        restaurants_info.append({
            'link': rest_url,
            'title': rest['title'],
            'address': rest_address
        })
        time.sleep(3)
    driver.quit()
    #print(restaurants_info)
    return restaurants_info

In [129]:
restaurants_info_list = get_resaurants_addresses(all_restaurants)

<ipython-input-128-068cd753f679>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [130]:
restaurant_df = pd.DataFrame (restaurants_info)

In [121]:
restaurant_df.to_csv('all_restaurants.csv',index=False)

# Yandex maps scarping

In [214]:
def getall_restaurants_yandex(chromedrive_path,url):    
    all_restaurants = []
    driver = webdriver.Chrome(chromedrive_path)
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #Scroll to Bottom of Webpage
    #driver.maximize_window()
    #scrollable_div = driver.find_element(By.cssSelector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show'))
    #driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)    #scroll_down_page()
    #time.sleep(10)
    all_restaurants_content = driver.page_source
    all_restaurants_response = Selector(all_restaurants_content)
    for el in all_restaurants_response.xpath('//ul[@class="search-list-view__list"]'):
        all_restaurants.append({
            'link': 'https://yandex.ru'+ el.xpath('./li/div/div/a/@href').extract_first(''),
            'title': el.xpath('./li/div/div/a/@aria-label').extract_first('')
        })
    driver.quit()
    return all_restaurants

In [215]:
yandex_maps_ulr  = 'https://yandex.ru/maps/2/saint-petersburg/search/%D0%93%D0%B4%D0%B5%20%D0%BF%D0%BE%D0%B5%D1%81%D1%82%D1%8C/?filter=alternate_vertical%3ARequestWindow&ll=30.280939%2C59.942188&sctx=ZAAAAAgBEAAaKAoSCRR5knTNUD5AEfTF3osv%2BE1AEhIJZVJDG4CN7z8RCOkpcoi40j8iBgABAgMEBSgKOABAAkgBYityZWFycj1zY2hlbWVfTG9jYWwvR2VvL0VuYWJsZUJlYXV0eUZpbHRlcj0xagJydZ0BzcxMPaABAKgBAL0Ba4J7%2BcIBhwHdhc%2FcIJzcwJSWA6%2BK7ta8BLvL6LsEoa%2Fjx8sF%2BJ%2Ft6dkC18fqg5kEscKN0wTpybW8DNqt%2BJgFl%2F%2BFmwSVwfOtsQXF2%2FGYQfqTzojuAavpuZoE7J%2FmqAb3urHA2wTmk8%2FQBKqlxtN6%2BLOezIUF%2B5XPwawGtdeI5AOdlYzjA4DB9u4DouvNswbqAQDyAQD4AQCCAhPQk9C00LUg0L%2FQvtC10YHRgtGMigJcMTg0MTA2MzkwJDE4NDEwNjM4NiQxODQxMDYzOTYkMTg0MTA2MzkyJDE4NDEwNjM4NCQxODQxMDYzOTQkMzUxOTMxMTQ5MzckMTM4Nzc4ODk5NiQ3NzA5MzE1MzeSAgCaAgxkZXNrdG9wLW1hcHM%3D&sll=30.280939%2C59.942188&sspn=0.283161%2C0.083994&z=12.8'
yandex_restaurants_list = getall_restaurants_yandex(chromedrive_path,yandex_maps_ulr)

<ipython-input-214-65bb33a6d908>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [216]:
yandex_restaurants_list

[{'link': 'https://yandex.ru/maps/org/koryushka/8784167645/',
  'title': 'Корюшка'}]

In [ ]:
https://yandex.ru/maps/org/koryushka/8784167645/